<a href="https://colab.research.google.com/github/shadialameddin/tensor_decomposition/blob/master/tensor_decomposition_test.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

http://tensorly.org/stable/auto_examples/index.html#general-examples

https://github.com/tensorly/tensorly

In [8]:
!pip install -U tensorly

Requirement already up-to-date: tensorly in /usr/local/lib/python3.7/dist-packages (0.6.0)


In [9]:
import numpy as np
import tensorly as tl
from tensorly.testing import assert_array_equal

A tensor is simply a numpy array

In [10]:
tensor = tl.tensor(np.arange(24).reshape((3, 4, 2)))
print('* original tensor:\n{}'.format(tensor))

* original tensor:
[[[ 0  1]
  [ 2  3]
  [ 4  5]
  [ 6  7]]

 [[ 8  9]
  [10 11]
  [12 13]
  [14 15]]

 [[16 17]
  [18 19]
  [20 21]
  [22 23]]]


Unfolding a tensor is easy

In [11]:
for mode in range(tensor.ndim):
    print('* mode-{} unfolding:\n{}'.format(mode, tl.unfold(tensor, mode)))

* mode-0 unfolding:
[[ 0  1  2  3  4  5  6  7]
 [ 8  9 10 11 12 13 14 15]
 [16 17 18 19 20 21 22 23]]
* mode-1 unfolding:
[[ 0  1  8  9 16 17]
 [ 2  3 10 11 18 19]
 [ 4  5 12 13 20 21]
 [ 6  7 14 15 22 23]]
* mode-2 unfolding:
[[ 0  2  4  6  8 10 12 14 16 18 20 22]
 [ 1  3  5  7  9 11 13 15 17 19 21 23]]


Re-folding the tensor is as easy:

In [12]:
for mode in range(tensor.ndim):
    unfolding = tl.unfold(tensor, mode)
    folded = tl.fold(unfolding, mode, tensor.shape)
    assert_array_equal(folded, tensor)